In [ ]:
from objectnat import get_intermodal_graph

# Importing the necessary libraries:
# - 'get_intermodal_graph' from 'objectnat' to work with intermodal graphs
# - 'geopandas' for handling geospatial data

G_intermodal = get_intermodal_graph(osm_id=1114252)
# Retrieving the intermodal graph for a specific region using OSM ID 1114252. This graph includes transportation networks
# (such as roads, public transport routes) for a given area.

In [ ]:
from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrones

# - 'get_accessibility_isochrones' from 'objectnat' to work with isochrones
# - 'geopandas' for handling geospatial data

points = gpd.GeoDataFrame(geometry=[Point(30.295606, 59.9439234)], crs=4326).to_crs(G_intermodal.graph['crs'])
# Creating a GeoDataFrame containing a point of interest (with longitude 30.295606 and latitude 59.9439234).
# The point is initially in EPSG:4326 (WGS84, commonly used for GPS coordinates), and then reprojected to match the CRS
# of the intermodal graph

isochrones, stops, routes = get_accessibility_isochrones(
    points=points,
    weight_type="time_min",
    weight_value=10,
    graph_nx=G_intermodal
)
# Calculating isochrones (zones of accessibility) from the specified points.
# - 'weight_type' is set to 'time_min', meaning the isochrones represent areas that can be reached within a specific time.
# - 'weight_value' is set to 10, indicating a maximum travel time of 10 minutes.
# - 'graph_nx' is the intermodal graph used to calculate accessibility.
# The function returns:
# - 'isochrones': polygons representing areas reachable from the points within the specified time limit
# - 'stops': public transport stops within the isochrones, if any
# - 'routes': public transport routes within or crossing the isochrones, if any

# Visualization
m = isochrones.explore(tiles='CartoDB positron')
stops.explore(m=m)
routes.explore(m=m, column='type')